In [4]:
%%capture
%pip install -r requirements.txt

In [2]:
from modules.stock_data import get_stock_data, load_dfs
from modules.dataset import gen_dataset
from modules.agent import DQNAgent
from modules.trainer import train
from modules.graph import training_loss, action_graph

from easydict import EasyDict as edict
import torch.optim as optim
import torch.nn as nn

In [3]:
CONFIGS = edict({
    "GET_DATASET": True,
    "TICKERS": [
        "KS11",
        "KQ11",
    ],
    "DATAFRAME_PATH": "./data",
    "WINDOW_SIZE": 10,
    "OUTPUT_SIZE": 2,
    "EPOCHS": 50,
    "GAMMA": 0.95,
    "BATCH_SIZE": 2,
    "LEARNING_RATE": 1e-5,
})

In [4]:
if CONFIGS.GET_DATASET:
    get_stock_data(CONFIGS)

df = load_dfs(CONFIGS)
dataset = gen_dataset(df)

agent = DQNAgent(CONFIGS)

loss_fn = nn.MSELoss()
optimizer = optim.Adam(agent.parameters(), lr=CONFIGS.LEARNING_RATE)

In [5]:
df[0]["Close"]

Date
2001-06-11     608.23
2001-06-12     607.15
2001-06-13     614.05
2001-06-14     613.75
2001-06-15     618.96
               ...   
2025-11-06    4026.45
2025-11-07    3953.76
2025-11-10    4073.24
2025-11-11    4106.39
2025-11-12    4148.78
Name: Close, Length: 6028, dtype: float64

In [6]:
dataset[0]

(tensor([608.2300, 607.1500, 614.0500, 613.7500, 618.9600, 608.7800, 608.9100,
         595.7200, 595.5300, 599.0800]),
 tensor(1.0600))

In [7]:
loss_history = train(agent, dataset, CONFIGS, optimizer, loss_fn)

Epoch 10/50
Epoch 20/50
Epoch 30/50
Epoch 40/50
Epoch 50/50


In [8]:
training_loss(loss_history)

In [9]:
actions = agent.actions(dataset)
action_graph("test", df[0]["Close"], actions[:len(df[0]["Close"])])